In [1]:
import load_model 
import torch
from board import Board
from gamestate import GameState
from features import Features

nn_chkpt = 'Network_Files/KataGo_NN/ckpt_files/kata1-b28c512nbt-s8834891520-d4763401477/model.ckpt'

kata_model, kata_swa_model, other_state_dict = load_model.load_model(nn_chkpt, use_swa=True, device = torch.device("cpu"))

In [2]:
colstr = 'ABCDEFGHJKLMNOPQRST'

def parse_coord(s, board):
    if s == 'pass':
        return board.Board.PASS_LOC
    return board.loc(colstr.index(s[0].upper()), board.y_size - int(s[1:]))

def str_coord(loc,board):
    if loc == Board.PASS_LOC:
        return 'pass'
    x = board.loc_x(loc)
    y = board.loc_y(loc)
    return '%c%d' % (colstr[x], board.y_size - y)

command = [['play'],['b'], ['B18']]

board_size = 19

gs = GameState(board_size, GameState.RULES_TT)

pos_len = 19

model_config = kata_swa_model.config
kata_model.eval()

features = Features(model_config, pos_len)

pla = (Board.BLACK if command[1] == "B" or command[1] == "b" else Board.WHITE)
loc = parse_coord(command[2][0],gs.board)
gs.play(pla,loc)

print(gs.board.to_string())

outputs = gs.get_model_outputs(kata_swa_model)
loc = outputs["genmove_result"]
pla = gs.board.pla
gs.play(pla,loc)
coord = str_coord(loc,gs.board)

print(coord)

print(gs.board.to_string())

. . . . . . . . . . . . . . . . . . . 
. O . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . * . . . . . . . . . . . * . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . * . . . . . . . . . . . * . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
Q4
. . . . . . . . . . . . . . . . . . . 
. O . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . * . . . . . . . . . . . * . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . 

In [31]:
def parse_moves(moves : list[str]):
    parsed_move = []
    for move in moves:
        split_move = move.split('[')
        player = split_move[0]
        pos = split_move[1].replace(']', '')
        parsed_move.append([player, pos])
    
    return(parsed_move)

def parse_metadata(metadata : str):

    split_data = metadata.split(']')

    board_size = split_data[2].split('[')[1]
    black_player = split_data[3].split('[')[1]
    white_player = split_data[4].split('[')[1]
    rules = split_data[10].split('[')[1]

    return(board_size, black_player, white_player, rules)

def read_file():
    with open('temp_data/5AD18C7E890130D6.sgfs') as sgf_file:
        sgf_lines = sgf_file.read().split(';')
        sgf_metadata = sgf_lines[1]
        sgf_moves_confidence = sgf_lines[2:]
        sgf_result = sgf_lines[-1].split('C')[1]
        sgf_moves = []
        for move_confidence in sgf_moves_confidence:
            sgf_moves.append(move_confidence.split('C')[0])
    
    board_size, black_player, white_player, rules = parse_metadata(sgf_metadata)
    mooves = parse_moves(sgf_moves)

read_file()